In [32]:

import os
import sys

import pandas as pd 
import numpy as np 

from analysis_tools.module_libraries import general_tool_module as tools
from analysis_tools.module_libraries import praise_tool_module as praise_tools


import holoviews as hv
from holoviews import opts
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt

import base64
from IPython.display import HTML

import scrapbook as sb

In [14]:
rounds=np.arange(7)+1
datadir = 'distribution_results'
allrounds_df = {}
for kr in rounds:
     allrounds_df['round-'+str(kr)]=pd.read_csv(f'{datadir}/round-{kr}/results/distribution/praise_outliers.csv')
    

In [46]:
allrounds_df['round-4'].keys()

Index(['index', 'ID', 'DATE', 'TO USER ACCOUNT', 'FROM USER ACCOUNT', 'REASON',
       'SCORE 1', 'SCORE 2', 'SCORE 3', 'SCORE 4', 'DUPLICATE ID 1',
       'DUPLICATE ID 2', 'DUPLICATE ID 3', 'DUPLICATE ID 4', 'DISMISSED 1',
       'DISMISSED 2', 'DISMISSED 3', 'DISMISSED 4', 'QUANTIFIER 1 USERNAME',
       'QUANTIFIER 2 USERNAME', 'QUANTIFIER 3 USERNAME',
       'QUANTIFIER 4 USERNAME', 'AVG SCORE', 'CORRECTION ADD',
       'CORRECTION SUB', 'CORRECTION COMMENT', 'FINAL QUANT', 'SPREAD'],
      dtype='object')

In [110]:
allrounds_finaldist = {}
for kr in rounds:
    allrounds_finaldist['round-'+str(kr)] = pd.read_csv(f'{datadir}/round-{kr}/results/distribution/final_praise_token_allocation.csv')

In [113]:
allrounds_finaldist['round-1'].columns

Index(['USER IDENTITY', 'USER ADDRESS', 'PRAISE_REWARD', 'QUANT_REWARD',
       'NR_OF_PRAISES_QUANTIFIED', 'REWARDBOARD_REWARD', 'TOTAL TO RECEIVE'],
      dtype='object')

# General statistics

In [17]:
round_stats = pd.DataFrame(index=allrounds_df.keys())

In [38]:
round_stats['round_start_time'] =  [allrounds_df['round-'+str(kr)]['DATE'].min()[:10] for kr in rounds]
round_stats['round_end_time'] =  [allrounds_df['round-'+str(kr)]['DATE'].max()[:10] for kr in rounds]

In [39]:
round_stats

,total_praise,round_start_time,round_end_time
round-1,599,2021-07-11,2021-07-30
round-2,1147,2021-07-31,2021-08-30
round-3,1432,2021-08-31,2021-09-29
round-4,1643,2021-09-30,2021-10-28
round-5,1661,2021-10-31,2021-11-27
round-6,1477,2021-11-28,2021-12-26
round-7,1731,2021-12-27,2022-01-30


## Praise involvement

### How many praise? 

In [45]:
round_stats['total_praise'] = [len(allrounds_df['round-'+str(kr)]) for kr in rounds]
px.line(round_stats,x='round_start_time',y='total_praise',markers=True)
#plt.xlabel(round_stats['round_start_time'].values)

### How many people give praise?

In [48]:
round_stats['total_praise_giver'] = [len(np.unique(allrounds_df['round-'+str(kr)]['FROM USER ACCOUNT'])) for kr in rounds]
px.line(round_stats,x='round_start_time',y='total_praise_giver',markers=True)


### How many people receiving praise?

In [56]:
round_stats['total_praise_receiver'] = [len(np.unique(allrounds_df['round-'+str(kr)]['TO USER ACCOUNT'])) for kr in rounds]
px.line(round_stats,x='round_start_time',y=['total_praise_receiver','total_praise_giver'],markers=True,title='total praise giver and receiver')


## Quantifier involvement

In [57]:
round_stats['total_quantifier'] = [len(np.unique(allrounds_df['round-'+str(kr)].filter(like='QUANTIFIER'))) for kr in rounds]
px.line(round_stats,x='round_start_time',y=['total_quantifier'],markers=True,title='total quantifiers')

# System health evaluation


## new TEC members

In [91]:
round_stats['round_user_list'] = [set(np.unique(allrounds_df['round-'+str(kr)].filter(like='ACCOUNT')))
            .union(set(np.unique(allrounds_df['round-'+str(kr)].filter(like='QUANTIFIER')))) for kr in rounds]

In [101]:
round_stats['round_user_new'] = np.nan
round_stats['round_user_new'].iloc[1:] = [len(round_stats.loc['round-'+str(kr+2),'round_user_list'] - 
                                         round_stats.loc['round-'+str(kr+1),'round_user_list']) for kr in np.arange(max(rounds)-1)]
round_stats['round_user_left'] = np.nan
round_stats['round_user_left'].iloc[1:] = [len(round_stats.loc['round-'+str(kr+1),'round_user_list'] - 
                                         round_stats.loc['round-'+str(kr+2),'round_user_list']) for kr in np.arange(max(rounds)-1)]

/Users/zhiweili/opt/anaconda3/envs/rad_env/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [105]:
round_stats['round_net_user_diff']=round_stats['round_user_new']-round_stats['round_user_left']

In [106]:
px.line(round_stats,x='round_start_time',y=['round_user_new','round_user_left','round_net_user_diff'])

## distribution equality

### Nakamoto Coefficient

The Nakamato Coefficient is defined as the smallest number of accounts who control at least 50% of the resource. Although its significance relates to the prospect of a 51% attack on a network, which may not be relevant in our context, we can still use it as an intuitive measure of how many individuals received the majority of a resource.

Bigger coefficient means more equal (i.e. needs more people to pass 50%), smaller means more concentrated power. The number should always be an integer

In [127]:
def nakamoto_coeff(x, key):
    sorted_x = x.sort_values(by=key, ascending=False)
    tot_sum = np.array(sorted_x[key].cumsum())
    try:
        winner = np.array([k for k in range(len(tot_sum))
                          if tot_sum[k] > 0.5]).min() + 1
    except:
        winner = -1
    return winner
def nakamoto_coeff_ratio(x, key):
    winner = nakamoto_coeff(x, key)
    ratio = winner / len(x)
    return ratio

In [122]:
for kr in rounds:
    totalreceive = allrounds_finaldist['round-'+str(kr)]['TOTAL TO RECEIVE']
    allrounds_finaldist['round-'+str(kr)]['PERCENTAGE'] =  totalreceive/ sum(totalreceive)

In [128]:
round_stats['nakamoto']  = [nakamoto_coeff(allrounds_finaldist['round-'+str(kr)],'PERCENTAGE') for kr in rounds]
round_stats['nakamoto_ratio']= [nakamoto_coeff_ratio(allrounds_finaldist['round-'+str(kr)],'PERCENTAGE') for kr in rounds]
px.line(round_stats,x='round_start_time',y='nakamoto',markers=True,title='minimum number of people to receive 50% of total rewards')


In [130]:
px.line(round_stats,x='round_start_time',y='nakamoto_ratio',markers=True,title='ratio of people to receive 50% of total rewards')


## quantification agreement

TODO: measure how well quantifiers agree with each other. metrics like ratio of agreement on duplication and dismissal. overall average spread. etc.